# Make Results Tables

This notebook generates full HTML tables of results for usage within the online documentation.

In [1]:
from plot_funcs import (get_results, get_ranks_sizes, get_across_ranks, clean_name,
                        get_intra_pipeline_df, clean_model_names, get_highest_performing_df)
from funcs import save_results_table, clean_col_names, save_table
from IPython.display import display

In [2]:
results = get_results('../exp/results')
static = {'random': True, 'base': True, 'fs': True, 'ico': True}

Found: 64530 Incomplete: 0


## Base Expiriment - By Parcellation

In [3]:
r_df = get_ranks_sizes(results, log=False, 
                       keep_full_name=True,
                       add_raw=True, **static)

# Show top result
display(r_df.sort_values('Mean_Rank').iloc[[0]])

# Show the two freesurfer
display(r_df[r_df['full_name'] == 'freesurfer destr'])
display(r_df[r_df['full_name'] == 'freesurfer desikan'])

# Save raw results
save_results_table(r_df, name='raw_results1')

,Parcellation_Type,Mean_Rank,Size,r2,roc_auc,full_name
194,Existing,46.703704,949,0.085201,0.629424,difumo 1024


,Parcellation_Type,Mean_Rank,Size,r2,roc_auc,full_name
11,Freesurfer Extracted,97.422222,150,0.073573,0.61954,freesurfer destr


,Parcellation_Type,Mean_Rank,Size,r2,roc_auc,full_name
10,Freesurfer Extracted,142.562963,68,0.061316,0.606063,freesurfer desikan


## Base Expiriment - By Pipeline

### Intra-Pipeline

In [4]:
# Get df
intra_pipe_df = get_intra_pipeline_df(results, log=False, keep_full_name=True, add_raw=True,
                                      threshold=False, **static)

# Get display col names and change order
intra_pipe_df = clean_col_names(intra_pipe_df)
intra_pipe_df = intra_pipe_df[['Parcellation', 'Pipeline', 'Mean Rank',
                               'Size', 'Mean R2', 'Mean ROC AUC']]

# Save
save_table(intra_pipe_df.sort_values(['Pipeline', 'Mean Rank']),
           'intra_pipe_table')

### Inter-Pipeline

In [5]:
# Get df
inter_pipe_df = clean_model_names(
    get_across_ranks(results, log=False,
                     **static))

# Prep
inter_pipe_df = inter_pipe_df.drop('Parcellation', axis=1)
inter_pipe_df = clean_col_names(inter_pipe_df)
inter_pipe_df = inter_pipe_df[['Parcellation', 'Pipeline', 'Mean Rank', 'Size']]

# Save
save_table(inter_pipe_df.sort_values('Mean Rank'), 'inter_pipe_table')

## Base Expiriment - By Target

In [6]:
# Get data
r_df = get_ranks_sizes(results, avg_targets=False,
                       keep_full_name=True,
                       log=False, threshold=False, **static)

# Get display col names and change order
r_df = clean_col_names(r_df)
r_df = r_df[['Target', 'Parcellation', 'Mean Rank', 'Mean Score', 'Size']]

# Save the table
save_table(r_df.sort_values('Target'), 'by_target_full_results')

## Full Results - Single + Multiple Parcellations

In [7]:
r_df = get_ranks_sizes(results, log=False,
                       keep_full_name=True, add_raw=True,
                       stacked=True, voted=True, grid=True,
                       **static)

# Show top result
display(r_df.sort_values('Mean_Rank').iloc[[0]])

# Save raw results
save_results_table(r_df, name='raw_full_results')

,Parcellation_Type,Mean_Rank,Size,r2,roc_auc,full_name
401,Voted,35.17037,5000,0.096004,0.641626,voted random 500 10 0


## Ensemble Only - By Pipeline

### Intra-Pipeline

In [8]:
# Get df
intra_pipe_df = get_intra_pipeline_df(results, log=False,
                                      keep_full_name=True,
                                      add_raw=True, stacked=True, voted=True)

# Get display col names and change order
intra_pipe_df = clean_col_names(intra_pipe_df)
intra_pipe_df = intra_pipe_df[['Parcellation', 'Pipeline', 'Mean Rank',
                               'Size', 'Mean R2', 'Mean ROC AUC']]

# Save
save_table(intra_pipe_df.sort_values(['Pipeline', 'Mean Rank']),
           'ensemble_intra_pipe_table')

### Inter-Pipeline

In [9]:
# Get df
inter_pipe_df = clean_model_names(
    get_across_ranks(results, log=False, stacked=True, voted=True, drop_all=True))

# Prep
inter_pipe_df = inter_pipe_df.drop('Parcellation', axis=1)
inter_pipe_df = clean_col_names(inter_pipe_df)
inter_pipe_df = inter_pipe_df[['Parcellation', 'Pipeline', 'Mean Rank', 'Size']]

# Save
save_table(inter_pipe_df.sort_values('Mean Rank'), 'ensemble_inter_pipe_table')

## Highest Performing Only

In [10]:
# Get scores df
r_df = get_highest_performing_df(results).reset_index()

# Prep for display
r_df = clean_model_names(clean_col_names(r_df))
r_df = r_df[['Parcellation', 'Pipeline', 'Mean Rank', 'Size', 'Mean R2', 'Mean ROC AUC']]
r_df['Parcellation'] = r_df['Parcellation'].apply(clean_name)

# Save the table
save_table(r_df.sort_values('Mean Rank'), 'best_of_full_results')